Getting all the tweets - opening them and export users ids in Ile de France

In [1]:
import os
import sys
import re
import gzip
import json
import numpy as np
from time import time
import multiprocessing as mp
import pandas as pd
import geocoder
import shapely
import twitter_api_functions

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
path_to_gzs = '../data/'
path_to_export = '../data/users/'
gzs = sorted([x for x in os.listdir(path_to_gzs) if x[-3:]=='.gz'])

# Get all tweets

On ouvre tous les tweets obtenus avec l'API Streaming en parallélisant (car très long).

In [3]:
def get_tweets(index_block,n_blocks):
    path_to_gzs = '../data/'
    gzs = sorted([x for x in os.listdir(path_to_gzs) if x[-3:]=='.gz'])

    tweets  = []
    for gz in gzs[n_blocks*index_block:n_blocks*(index_block+1)]:
        print(str(gz))
        with gzip.open(path_to_gzs+gz,'rb') as f:

            for line in f:
                try:
                    tweet = json.loads(line.decode("utf-8"))
                    list_columns=[
                    tweet['id_str'],
                    tweet['user']['id_str'],
                    tweet['user']['name'],
                    tweet['text'],
                    pd.to_datetime(tweet['created_at']), 
                    tweet['lang']]

                    if (tweet['place'] is not None): 
                        list_columns.extend([tweet['place']['country'],
                                             tweet['place']['country_code'],
                                             tweet['place']['place_type'],
                                             tweet['place']['name'], 
                                             tweet['place']['full_name'],
                                             tweet['place']['bounding_box']['type'],
                                             tweet['place']['bounding_box']['coordinates']])
                    if (tweet['coordinates'] is not None) :
                        list_columns.extend([tweet['coordinates']['coordinates'][1], tweet['coordinates']['coordinates'][0]])
                    else :
                        list_columns.extend([np.nan, np.nan])

                    tweets.append(list_columns)
                except:
                    print("error "+str(IOError))
                    pass
    return pd.DataFrame(tweets, columns=['_id','user_id','user_name','text','time','language','country','country_code','place_type','city', 'full_name_place','bounding_box_type','bounding_box_coordinates','latitude','longitude'])



In [4]:
# START POOL OF CPUS
try:
    n_cpu = int(os.getenv("SLURM_CPUS_PER_TASK"))
    print('INFER # CORES FROM PBS VARIABLE')
except:
    n_cpu = mp.cpu_count()
    print('USE DEFAULT # CORES')
    
pool  = mp.Pool(processes=n_cpu)
print("# PROCESSORS:", n_cpu, "\n")

print("Read Tweets ...")
start = time()

# COMPUTE LIST OF RESULTS
n_blocks= len(gzs)//n_cpu + len(gzs)%n_cpu
print(n_blocks)
results = [pool.apply_async(twitter_api_functions.get_tweets, args=(index_block,n_blocks)) for index_block in range(n_cpu)]
print(results)
tweets  = pd.concat([results[index_block].get() for index_block in range(n_cpu)]).reset_index(drop=True)
#print(tweets)

pool.close()    
print("DONE IN", round(time() - start), "SEC")


USE DEFAULT # CORES
twitter-stream-geobox-IDF-2020-04-13-03.json.gz
# PROCESSORS: 16 

Read Tweets ...
2
[<multiprocessing.pool.ApplyResult object at 0x7f433c676d90>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c050>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c110>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c1d0>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c290>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c3d0>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c490>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c550>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c610>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c390>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c710>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c7d0>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c890>, <multiprocessing.pool.ApplyResult object at 0x7f433c67c950>, <multiprocessing.pool.ApplyResult object

# Get tweets in Paris or Ile-de-France

On fait une vérification : il s'agit bien des tweets à Paris ou en Ile de France

In [5]:
# Polygons Paris and Ile-De-France
polygon_paris = Polygon([[2.22412196,48.81557548],[2.46976017,48.81557548],[2.46976017,48.90215593],[2.22412196,48.90215593],[2.22412196,48.81557548]])
polygon_idf = Polygon([[2.402662938565835, 48.32071765323825], [2.398482376542098, 48.318696362741775], [2.396853400910401, 48.31475529806563], [2.388870961591493, 48.312399311894325], [2.385251717307108, 48.311500618596824], [2.381342813924157, 48.31150274722007], [2.375343332237395, 48.31024578144871], [2.369950791588418, 48.30867412924205], [2.35593443948718, 48.31034060947027], [2.355799806489728, 48.31043075166589], [2.356473461502361, 48.31104354715907], [2.355161555951771, 48.313537711757796], [2.353474281481295, 48.31440306425807], [2.348822966141809, 48.31584685861426], [2.340155093977896, 48.32023011757676], [2.330864089214374, 48.326714615086026], [2.328166975376719, 48.326399728055016], [2.32857108039881, 48.33089532909492], [2.328706282576607, 48.33161498766452], [2.329312870154224, 48.332828573765816], [2.327694800252112, 48.33305333064347], [2.324222539351009, 48.332131128314266], [2.318121791969129, 48.33017520942476], [2.317514546264567, 48.330804229236136], [2.317176744461912, 48.331568861080264], [2.312593571187, 48.330803656999464], [2.304374649268758, 48.319787755945164], [2.300737999889984, 48.31569585677213], [2.299728494718988, 48.31421159467425], [2.298651549700812, 48.312224859144685], [2.298719296059887, 48.31199942194104], [2.300202373939027, 48.311236208825115], [2.297911927945223, 48.309526604582025], [2.296227773462451, 48.30858265465945], [2.295419291196409, 48.308222440340124], [2.290229303770435, 48.30952397394467], [2.287047298193118, 48.31073661890155], [2.277811570854005, 48.31273297749078], [2.273499394477815, 48.313113280842344], [2.269049358510904, 48.3141889882934], [2.269115740135252, 48.31504326728641], [2.266892338648934, 48.31468221267722], [2.25888367236765, 48.30721472103461], [2.254849426150754, 48.301412900426996], [2.253570013214852, 48.30024323816268], [2.245151884831329, 48.29839390338474], [2.24661624752266, 48.308869316725726], [2.249436070081645, 48.31458091702606], [2.240335361674118, 48.31632688183027], [2.238042838203596, 48.316370632558495], [2.239722617468447, 48.31933893620536], [2.24483112179043, 48.32726337110624], [2.246310391108282, 48.32996214885505], [2.245229597952229, 48.33146774225998], [2.243135379349591, 48.332971350427385], [2.239177446478493, 48.331395721230834], [2.23431417866907, 48.327884139622064], [2.231820002976367, 48.32765744274041], [2.22959232112012, 48.329229232044966], [2.228370521949908, 48.33295883909374], [2.223913790047606, 48.33641623994784], [2.21960073492719, 48.33551334176705], [2.21839006310024, 48.33407295717949], [2.214694886102563, 48.3365462139133], [2.212576498576572, 48.34049553390465], [2.207165339280995, 48.341793757780486], [2.207090946585399, 48.34494056618432], [2.204916423660642, 48.34486120800848], [2.202252044121844, 48.34459856281837], [2.201875492557901, 48.34308445480324], [2.198209769113149, 48.33706287789637], [2.191476441016595, 48.33385447341525], [2.185025102223043, 48.32642865381036], [2.183487864363201, 48.322020936118065], [2.184569822129176, 48.320629263498404], [2.1839726026086, 48.31772881076467], [2.181489564963528, 48.313791389531104], [2.171709963523325, 48.3159358919047], [2.170837855410007, 48.31503631779172], [2.169629252455773, 48.31323640751741], [2.169025380691481, 48.312695907528884], [2.163832233859416, 48.313542011462246], [2.15438962622473, 48.315910610684114], [2.152300126041117, 48.315907363432906], [2.151835665407345, 48.313839064636], [2.154352312414407, 48.30714476735912], [2.155708478858807, 48.30449381337798], [2.156585634029772, 48.304293118971096], [2.160964762225487, 48.30454738937316], [2.16184611420797, 48.303064505078204], [2.162187965703454, 48.30158229377325], [2.162527108973685, 48.30086307262014], [2.1636100598563, 48.29924663167886], [2.163748326876274, 48.29843690799612], [2.16031093710084, 48.29852234047267], [2.15651264320523, 48.29908108482554], [2.145081453296227, 48.299307679741204], [2.136119995833762, 48.29924799931547], [2.126152214901647, 48.29823184151332], [2.110594467838899, 48.296944412570355], [2.112257043778369, 48.30207307386812], [2.113716899114126, 48.3072451614573], [2.111224473391296, 48.307240868080925], [2.106168036689448, 48.30763558473594], [2.104619657658776, 48.30731795603244], [2.100987639264507, 48.30564742251419], [2.099845479803708, 48.30492521352501], [2.093801077442082, 48.300507930696114], [2.089379020829376, 48.29528310405464], [2.088169608130591, 48.29465113535371], [2.085343528223101, 48.29401566608415], [2.081505014981655, 48.29360352065049], [2.077596457772156, 48.293729680339744], [2.066723242267118, 48.294239183632826], [2.052713822663637, 48.295470353989415], [2.052179037557183, 48.29483981377458], [2.05191313834802, 48.29402966660645], [2.051930380007918, 48.290973192755786], [2.051462806097165, 48.290207315704244], [2.049377039948197, 48.28975327350414], [2.034498048440887, 48.288222801648296], [2.02284477164865, 48.288080340450726], [2.015919348440641, 48.285948886997375], [2.007237571011955, 48.28468860637469], [2.004677141608521, 48.284770570655795], [1.995370421082554, 48.28654281402998], [1.994090144531071, 48.286584124462344], [1.9911874691435192, 48.28756449640718], [1.990243837159493, 48.287696816756856], [1.980722518232461, 48.28705728879837], [1.975291586655453, 48.287202034768534], [1.972724877935009, 48.28821905881756], [1.973251403371509, 48.28998275961335], [1.9745117813654611, 48.29290907217038], [1.980377957732905, 48.292252519144995], [1.982376544631459, 48.295450540607476], [1.98028418279595, 48.29598320894394], [1.9740839356578732, 48.29639211607255], [1.96661022195828, 48.29558204343147], [1.9631313213394512, 48.2997519087821], [1.96114143068209, 48.30478055848723], [1.959225976988035, 48.30868506445257], [1.969441339340217, 48.31423171808394], [1.973464981656501, 48.31590880348917], [1.974141916322555, 48.315160644810945], [1.979071797619883, 48.3138724707825], [1.979983431571299, 48.3184827670238], [1.974491950838203, 48.32316372007222], [1.974489795723303, 48.32334365907896], [1.978716570476668, 48.326278408159865], [1.9823432207772362, 48.32831255507804], [1.974947303551144, 48.33525806596195], [1.975354207353962, 48.34479022742891], [1.9733953665972621, 48.35297468334298], [1.9824516450338252, 48.3601951537698], [1.986823728177806, 48.36214101576579], [1.989039974782607, 48.363586065798316], [1.986474995621458, 48.36398344129576], [1.985459126703853, 48.36442912605466], [1.984577953700509, 48.36510089625338], [1.980917926608973, 48.3701621047841], [1.979191995900595, 48.37740646546595], [1.976310838348386, 48.380270715937684], [1.970235615832931, 48.38047631867583], [1.9667276905253592, 48.380420597020716], [1.966381938602989, 48.38158815853024], [1.9731034423096512, 48.3931174397929], [1.976442525721306, 48.3983513080004], [1.977246874046626, 48.39916261408073], [1.9765710225943072, 48.39938531576533], [1.972672978870017, 48.401666629653214], [1.96222438942051, 48.40432164183499], [1.942770320741082, 48.405651458199536], [1.93548975680755, 48.40423401800707], [1.931037278362236, 48.40376928719261], [1.93042830575777, 48.40394644343977], [1.930561117871898, 48.40412668971786], [1.9305823153905992, 48.40846542391323], [1.9258894015480252, 48.41274175064107], [1.92735282187494, 48.41566833490979], [1.932129764112304, 48.42010212694179], [1.937806831512828, 48.42639000317327], [1.9373240561925311, 48.427534627401066], [1.936764861135698, 48.430049741197834], [1.9368227618237732, 48.43126398458675], [1.938516073640371, 48.43524780923887], [1.942158818518703, 48.44027196492608], [1.9428974719413161, 48.440993819715715], [1.940866052255318, 48.44161630267396], [1.94032429191042, 48.44165915106115], [1.938216349672905, 48.44131461295032], [1.933124841071103, 48.44226403247175], [1.93145013457681, 48.44482034632464], [1.930423413009713, 48.45340332691669], [1.930623940931089, 48.453718231629864], [1.929404330990824, 48.45416338903311], [1.922959227943058, 48.457467286345256], [1.922146278491287, 48.45759936196724], [1.922150081248724, 48.457150443493525], [1.922910258577549, 48.45504033474234], [1.9232608414020422, 48.45337815688975], [1.923196098028056, 48.452973600599464], [1.920737671762487, 48.4477500461492], [1.9118859071283292, 48.44776369238195], [1.906554981304499, 48.445672344528994], [1.9064076592587362, 48.44014671901166], [1.9042587645853142, 48.43860186947979], [1.9039212742019611, 48.4385105799107], [1.90142170742148, 48.43854684697296], [1.8983744774862932, 48.43942281724884], [1.8931646360806442, 48.44031364732221], [1.88478958809622, 48.43996711818918], [1.879446962687042, 48.44066625869525], [1.877488706859119, 48.44061356165377], [1.8637923949992081, 48.44446989551879], [1.861090665126532, 48.44445929373855], [1.860946771419584, 48.44535763660713], [1.860744672959385, 48.44563461101788], [1.857622782280466, 48.44678323932509], [1.852151412561482, 48.44664047586419], [1.847635648067589, 48.447571116614185], [1.844863060371809, 48.44936070653789], [1.8393315967173671, 48.45749600784709], [1.83850657406856, 48.459020158527075], [1.8363839553347492, 48.46647780689266], [1.825254440705768, 48.466560747035], [1.8114681706319882, 48.46626608846761], [1.807693754814629, 48.46584046567359], [1.802684715117166, 48.465867432498776], [1.801452315963026, 48.46604123961369], [1.802255376716872, 48.47015835415503], [1.7990578344462662, 48.475380391072], [1.7964958763057912, 48.47658076241365], [1.794460127357965, 48.477960965722495], [1.792889641487215, 48.4802302797639], [1.793712130604617, 48.48242223266885], [1.795258480191324, 48.48332864015738], [1.796330928957298, 48.48418793826782], [1.78563983668212, 48.490072057082955], [1.78697155816701, 48.49201103116317], [1.787841664952783, 48.49295920320578], [1.789382651008204, 48.49422460377765], [1.78998111244886, 48.49521587381591], [1.7906408745169662, 48.49683755363387], [1.790635203664074, 48.49737731543375], [1.789050848158861, 48.500156539064434], [1.782015095800177, 48.5034278935755], [1.77861704320384, 48.508131705449394], [1.777128131060337, 48.51641783904127], [1.77575430181074, 48.52770282718838], [1.77647848229939, 48.52963919548504], [1.778661586811004, 48.53436875195684], [1.78415861937927, 48.54506999065282], [1.78724230065562, 48.55374633613008], [1.778182006639273, 48.552625290379844], [1.778848358120056, 48.55357228016011], [1.776662016025811, 48.555449894682766], [1.767956285712767, 48.55895867295484], [1.767410428736174, 48.559405546049966], [1.763555710914647, 48.56383683770015], [1.762729388561109, 48.56513634549654], [1.762941392603099, 48.56729502215198], [1.765322855182955, 48.56937456840283], [1.757892670061779, 48.573788285593864], [1.755445405653792, 48.57454000952256], [1.7509027193045599, 48.57509517181456], [1.745862924839328, 48.57613703161486], [1.743299095851378, 48.572862236785646], [1.73683026060763, 48.57228093061955], [1.7140177821929772, 48.57683801399023], [1.709248676777636, 48.578026100982534], [1.710293340065278, 48.58135805241525], [1.701983609829003, 48.584999462078464], [1.705054772875644, 48.58879165771136], [1.718123877846951, 48.607172426861375], [1.718119655627905, 48.608674766939465], [1.717486849107729, 48.6104726118497], [1.71594566284638, 48.61312880063433], [1.7148984609516, 48.614392276566484], [1.7139604105699, 48.614524290347696], [1.709200739575768, 48.613649629198626], [1.689422105098955, 48.61153625035312], [1.688495117575624, 48.61165918940793], [1.688032058674677, 48.61151374499407], [1.68589783360283, 48.61238444694744], [1.684294509143637, 48.61350567879686], [1.683621384276087, 48.61399843576872], [1.68231531596098, 48.61559613342522], [1.68383688057991, 48.61616623498616], [1.685623827206012, 48.61611130294532], [1.6909307449948132, 48.617125169338465], [1.690633465649499, 48.617401685979175], [1.686690114693879, 48.619146726633076], [1.6839118808351712, 48.61949749837385], [1.679359138514777, 48.618488989605815], [1.676079584706227, 48.616825138709196], [1.666222999364638, 48.61370406332773], [1.656339110342003, 48.62205312113622], [1.656646797147471, 48.624446015694005], [1.657711131857219, 48.62607015404167], [1.658232568005397, 48.627645882716784], [1.655691336308519, 48.63010380054745], [1.649560658609581, 48.6322709259806], [1.6493016935889662, 48.635415780712506], [1.650428885376003, 48.6374446128546], [1.651571907013921, 48.638125076128475], [1.648612687996046, 48.63891739775663], [1.645375156886907, 48.6400222495115], [1.640990019929975, 48.6433899702156], [1.641951710607864, 48.64755267573591], [1.643494661261982, 48.64888818932813], [1.643682144057939, 48.650035533773135], [1.643190674026713, 48.65129119812029], [1.640480303438982, 48.65109477718989], [1.638304461476833, 48.650497777648944], [1.629847637980852, 48.64887508549489], [1.6219950813792732, 48.6501537524457], [1.617051997622839, 48.651989284988105], [1.615616032782937, 48.65278946494963], [1.610763052177883, 48.65694049942165], [1.608596897260806, 48.65899424075571], [1.603121101317205, 48.66242159122575], [1.6024917313047542, 48.66376543236528], [1.6056545170577259, 48.67676131442761], [1.605863327791321, 48.67876267068712], [1.609333685735536, 48.682918862097154], [1.610131381364711, 48.684137220261945], [1.610657587390978, 48.68539918739534], [1.611157650596602, 48.6884132530669], [1.611774872467951, 48.68947386183938], [1.59935732435896, 48.69175666275], [1.586950759831337, 48.69504945120395], [1.582183735345793, 48.69605259648432], [1.582315145770881, 48.696368116233955], [1.5795403954021912, 48.701811808772526], [1.5847567790305441, 48.70501336947122], [1.591712054054903, 48.70771796675158], [1.594145696207383, 48.70845194238866], [1.594881057985386, 48.709310708939405], [1.5899699129579599, 48.71072627584964], [1.5895382120500612, 48.71246815496513], [1.593365963720156, 48.715503537185946], [1.598596459632709, 48.72023746447686], [1.611285518291639, 48.730908910139846], [1.617804540369038, 48.735942039083845], [1.620250861457615, 48.73600182617016], [1.621519975626632, 48.73753673394008], [1.621635851451812, 48.73902053339159], [1.622707655130899, 48.74015104596198], [1.622364486191753, 48.7403737727173], [1.621799441790158, 48.74194326123594], [1.621996842351578, 48.742348537862085], [1.626403500442435, 48.748091646605765], [1.624964179842268, 48.7490276948727], [1.623049634559051, 48.74982511372757], [1.612682812960972, 48.7563239378886], [1.6026875153091882, 48.76048650622883], [1.582070645837061, 48.76183208560205], [1.582809665388563, 48.762376084631406], [1.584904586180228, 48.763296867933065], [1.584282277953444, 48.76396684589185], [1.582924910987864, 48.7675533982692], [1.587597326799866, 48.77360593190138], [1.5830773218484109, 48.7758237180617], [1.580399297487683, 48.77755949987972], [1.576624567245795, 48.78301879608003], [1.575674299705779, 48.790504096805606], [1.5782088093871982, 48.79861013891455], [1.583424089980054, 48.807168828466104], [1.582323734911496, 48.80797073493032], [1.58053137379387, 48.80867806325702], [1.579438671248947, 48.808896255423605], [1.579571574025022, 48.80916678663412], [1.580707409818234, 48.81056641326515], [1.583224040121389, 48.81067272403903], [1.59120588113724, 48.814866973856276], [1.590475898545971, 48.81733378465776], [1.589491648670033, 48.81952953780115], [1.585118326879177, 48.82543430594993], [1.584081408683483, 48.826595866863755], [1.578989180408092, 48.829439914251424], [1.5800653177164081, 48.83034588828318], [1.5837280523171149, 48.83213072656879], [1.594181784260237, 48.834218536497566], [1.5966251849889659, 48.837087789890994], [1.595191988192826, 48.83970733488273], [1.5896267828715942, 48.84227910983444], [1.58347381397579, 48.84399276405598], [1.5821773699652222, 48.84420946967167], [1.577272748395865, 48.844358116541045], [1.5772570720754642, 48.845481376049094], [1.5777045310248612, 48.84746194410668], [1.578292414544568, 48.84909248162585], [1.5815974879282062, 48.85499985317578], [1.58144666182535, 48.86063950990152], [1.578039310249433, 48.8608644098191], [1.570680872038293, 48.864682211526414], [1.566639574173438, 48.86618364792374], [1.564796276572091, 48.86653123493865], [1.559883648882892, 48.867038880631384], [1.558836128245632, 48.864246029482146], [1.5552126063722822, 48.86507571749671], [1.55161552860795, 48.86833326909435], [1.5467816162512582, 48.87216574370805], [1.5462328761118, 48.87243161638284], [1.547424322192556, 48.874641662202414], [1.5483587035170672, 48.875950915382816], [1.548699543990638, 48.87595324955156], [1.55414262396227, 48.87867251329735], [1.559507905489312, 48.88205527350731], [1.563817972550896, 48.89004646557699], [1.563740490691094, 48.890630173587425], [1.557253006645874, 48.891575743200214], [1.5545570035118952, 48.893221587013876], [1.550453492092882, 48.89749739189082], [1.547823339321779, 48.89955132405479], [1.544822612827508, 48.90133536098985], [1.5430091707374012, 48.901979795260324], [1.5407474749527061, 48.90391482909589], [1.5396732496563672, 48.90509888434219], [1.538523533551438, 48.90668585408314], [1.538428049146493, 48.908482656116455], [1.5417973746938531, 48.915479906681284], [1.540875811773005, 48.91680882954995], [1.540332009522659, 48.916686178886486], [1.5405245568914872, 48.918481977527414], [1.538617779792087, 48.92179469238781], [1.528488294045399, 48.92419808518992], [1.524102187536562, 48.924752145608885], [1.516481447908749, 48.92366686401995], [1.513919466815541, 48.921806580937165], [1.512895418883692, 48.92258536982847], [1.508450427576047, 48.930476526360586], [1.509013998363198, 48.93662746482592], [1.502486952571831, 48.940654611980015], [1.501526601311318, 48.941051842104244], [1.510998550313515, 48.95357450002123], [1.510191109188135, 48.95330004464622], [1.5010590902862, 48.952678048625486], [1.497912331428151, 48.956943203273916], [1.498515524694922, 48.95762171174711], [1.499428949083045, 48.96023420920854], [1.4993569242845761, 48.960458157222156], [1.496650409489568, 48.96232647829922], [1.4944530348555212, 48.96320999677112], [1.492648815291008, 48.96499421619758], [1.496605895722333, 48.969344788929554], [1.505384632371702, 48.97256640220803], [1.511766513633146, 48.97517303401673], [1.51554855685836, 48.976406187812934], [1.518385308415246, 48.97835784110935], [1.514001807945839, 48.97841783479837], [1.5124285278489942, 48.978631272312775], [1.509016549602727, 48.98283127636916], [1.507908822012272, 48.983767244361985], [1.504455028874754, 48.98203610257949], [1.49741602208099, 48.97955128481473], [1.484203624470962, 48.979681840629326], [1.477093193215915, 48.978791319260374], [1.4731974671826251, 48.9756847686946], [1.470892227020633, 48.974814469687814], [1.470609967254657, 48.97530643548572], [1.460920237110517, 48.98643383863926], [1.46091108179195, 48.986927482982466], [1.461698232464998, 48.9888658558887], [1.467765169173556, 48.9894498470854], [1.478751290586156, 48.99888468286422], [1.479570612445008, 49.00012630337889], [1.480462775592713, 49.0019165532252], [1.480556031691081, 49.004523196187435], [1.480098793157688, 49.00739536882735], [1.480029224252591, 49.00753029143511], [1.474050381504163, 49.00968843709832], [1.474450123522357, 49.010275405510164], [1.474125032309792, 49.01431389281479], [1.4721822976396601, 49.0184909723705], [1.470052996842526, 49.01915006031272], [1.461562667999601, 49.023491555239865], [1.458937495203411, 49.02513427846128], [1.457757007677809, 49.0262945720077], [1.457879338478311, 49.027148727642626], [1.457953358463872, 49.03483279820246], [1.456143772269515, 49.03676291933927], [1.453984088752085, 49.03831797617718], [1.450023039703447, 49.042197034816034], [1.447428696963269, 49.04518889807181], [1.448422721224942, 49.04699322066346], [1.449151059639485, 49.05027350332847], [1.447285110430396, 49.053509472820025], [1.45784229377846, 49.05979684928302], [1.460801236416761, 49.06273959824586], [1.4643810541043272, 49.0616873377584], [1.468930164702917, 49.05996744254014], [1.471827773848495, 49.05841613648232], [1.4750373877038832, 49.05453048495064], [1.478505653502417, 49.05235313514614], [1.479535439203403, 49.052090981092476], [1.484876688040105, 49.05154513169235], [1.487654237726856, 49.053047826945495], [1.4877163351541172, 49.05345240387421], [1.489129857269158, 49.05476611073531], [1.497113609674272, 49.05842125390023], [1.503294447404727, 49.0590982567285], [1.509498999435909, 49.06872063797463], [1.511600605828052, 49.07413619319277], [1.511343157559446, 49.07852786329206], [1.510085100719551, 49.0824826505923], [1.50801436049249, 49.08379241121543], [1.509573947155943, 49.08450061520548], [1.5112947670803312, 49.083838480328865], [1.5178955537679242, 49.073909130389914], [1.520411701958988, 49.06958213745925], [1.521346125694064, 49.06834810860814], [1.527603046448776, 49.06996453903425], [1.535757923402458, 49.0718070179871], [1.5465195506926421, 49.07238101324585], [1.547342519082409, 49.07266255020664], [1.549472549108642, 49.07255788191662], [1.551672589126794, 49.07180832882282], [1.553263658529864, 49.07056129172387], [1.5570743762241719, 49.06962058645414], [1.559362205370245, 49.06979192294931], [1.561066810921139, 49.07011720639533], [1.56924006175013, 49.073270577457414], [1.570515891830748, 49.07480645181779], [1.570578661495473, 49.075210977186195], [1.5698863307313289, 49.07574590833022], [1.569950088048891, 49.076114474107534], [1.570354730260005, 49.076432238919246], [1.572265045616378, 49.07680386971289], [1.573104348267105, 49.07784261595105], [1.5747375500156742, 49.07843753457369], [1.59033611067448, 49.08137508241763], [1.594352543875402, 49.08274764951237], [1.594969558509524, 49.082662238403856], [1.604486635031416, 49.083036595012544], [1.6087988076030322, 49.077894185219144], [1.610094572685814, 49.078171765026774], [1.62203581363385, 49.085172480323116], [1.6233233619235392, 49.08607840434069], [1.619126021003548, 49.087760445560484], [1.617534466724497, 49.09399651414702], [1.619528311118036, 49.09702743804031], [1.6212261003787192, 49.09914937528694], [1.6238616975359599, 49.10154639864524], [1.6259710858424712, 49.102413117853686], [1.6270460553159052, 49.103865992446764], [1.627027807215077, 49.10512383106815], [1.628646655483102, 49.106885940276314], [1.637837154557872, 49.115485504670225], [1.647091001922174, 49.12471353091347], [1.65002566157325, 49.125403831236], [1.655983941548498, 49.13038928200034], [1.652406172816969, 49.13944932248262], [1.652821590139087, 49.14165703897474], [1.653922530211235, 49.14544063416982], [1.655448064273778, 49.14744852551519], [1.658939019050382, 49.15037211215246], [1.664379261417072, 49.153368483463346], [1.664302518791416, 49.15395210956023], [1.663469253382308, 49.15484565321257], [1.662918625640383, 49.16024279002651], [1.664134669965497, 49.16375413455577], [1.667449443885073, 49.167510796113305], [1.6663294598757141, 49.176174580533065], [1.667016860186123, 49.1786135590155], [1.671623610382195, 49.18012201657547], [1.671823266262519, 49.180572904820046], [1.670203984483881, 49.189198730692716], [1.670538434261193, 49.189903769351105], [1.6712820112763151, 49.19059713494898], [1.67517175266838, 49.197276661177995], [1.676955222005737, 49.20254281272862], [1.676878770672876, 49.203171377955805], [1.676322327566359, 49.20384232577594], [1.6732001714160591, 49.20562157043371], [1.673647216966261, 49.20814949707728], [1.675742300719089, 49.21186711806247], [1.678786350365957, 49.213748261361864], [1.683774317187064, 49.2163231204367], [1.689762300409958, 49.219971409664865], [1.695199086803469, 49.224008130498646], [1.702558455549785, 49.228000550063705], [1.704064348373743, 49.22932047260418], [1.704497623769025, 49.23197303470082], [1.704358828824116, 49.23219722178433], [1.708064413609025, 49.235496279397], [1.7091178465679442, 49.23680408322277], [1.709925340217946, 49.238120033847046], [1.7100500713242321, 49.239064535347765], [1.7093926492107552, 49.24130707225104], [1.7126322213523242, 49.24118975122992], [1.713087714844929, 49.237620653417686], [1.712800062200277, 49.233015022753264], [1.713523920836379, 49.232672093602716], [1.729662120604027, 49.2291973031622], [1.74050425188046, 49.224385702831746], [1.740447618812849, 49.22344193576176], [1.735114419191794, 49.221933113275504], [1.734109178473332, 49.22127789039124], [1.733682376684575, 49.21095862367712], [1.7301084057779792, 49.20962716462932], [1.723039453795312, 49.209679245882576], [1.718956951166833, 49.207007741120236], [1.715576128821529, 49.20311803793349], [1.7155490546111132, 49.20266098505281], [1.718401493090029, 49.20278486868381], [1.720604836979805, 49.20224542099499], [1.724325759392109, 49.20070328163278], [1.7250290786240332, 49.19916779618548], [1.725574957462809, 49.19689244094348], [1.72582490451608, 49.19571270712963], [1.725871050702368, 49.19417295534453], [1.726855811934765, 49.194129417291514], [1.726826083778181, 49.194891580465246], [1.729133930710099, 49.195218960044386], [1.7352859208877032, 49.19504902146635], [1.737435706885569, 49.1946031151943], [1.7398899652796742, 49.19207523465231], [1.740455183190461, 49.190553492743774], [1.741934596216518, 49.18545188400663], [1.7417854466974192, 49.181317809071835], [1.742141599065365, 49.180151725268274], [1.742693405435824, 49.179795126330475], [1.755170590325829, 49.17454596946938], [1.775915094297692, 49.18369349722682], [1.7768014503926062, 49.18414668106554], [1.7763800318836642, 49.1850883042368], [1.776926849285914, 49.18522513338207], [1.777555486052915, 49.184104882999456], [1.790979178781881, 49.179854330039696], [1.7943306942813142, 49.18040853444105], [1.793679629962132, 49.183685700544615], [1.795474088156161, 49.18526277583291], [1.802576018216512, 49.184937962845176], [1.80303819370878, 49.18310290842928], [1.8107375241763832, 49.179088964693555], [1.814397964508514, 49.17616236295644], [1.8194069274323792, 49.17692549366297], [1.821794559726533, 49.17801436430202], [1.826575831534847, 49.17951675885872], [1.8322460241610772, 49.17585750373018], [1.837081878254271, 49.171698986359544], [1.83406368181637, 49.16511857596746], [1.834612737070771, 49.16503127850761], [1.838162888576254, 49.16630372349293], [1.845543383822084, 49.16989668734971], [1.851028559906948, 49.17082042447291], [1.8582162887735012, 49.16985387383768], [1.8587641322496542, 49.16994628451057], [1.868009720175445, 49.17186960719831], [1.876766515190267, 49.175304500931546], [1.8770421520919882, 49.17144885765527], [1.881018530582249, 49.167344416407715], [1.8826909756780772, 49.163614999178535], [1.8825644159278072, 49.16249078712817], [1.885234879118075, 49.16263602852517], [1.886805382525222, 49.16318105727831], [1.889876664465072, 49.164586009247444], [1.894536229716044, 49.1660405386233], [1.904657765577711, 49.168197469755725], [1.917431758176779, 49.171554382811166], [1.926182230329987, 49.17333029535235], [1.931476176782029, 49.174167721442096], [1.93385564991122, 49.17349468085774], [1.9343315348269132, 49.17356003394394], [1.934793666105032, 49.17289426921304], [1.9428186733731052, 49.17108310353733], [1.948017142087212, 49.1707491955362], [1.9530447231950232, 49.1714709563249], [1.9607171629873372, 49.173517298233655], [1.961877417502507, 49.17405958907339], [1.962696229106939, 49.17460162510678], [1.9642601005972922, 49.176358489503635], [1.966713494347146, 49.178217372353345], [1.971012622207819, 49.18079149858731], [1.9733655833962582, 49.18310343007821], [1.9734431640564312, 49.1838944150327], [1.983122907576111, 49.18067047068576], [1.996823915180955, 49.17645341531909], [1.999346284718758, 49.17586839275406], [1.999122083428424, 49.17540618014343], [2.000609626575429, 49.17571164673034], [2.002387091479497, 49.17634497780758], [2.004502173948826, 49.177609556498055], [2.021785875071654, 49.18867177015305], [2.024869584634746, 49.188814554272184], [2.038228896658568, 49.192137139630645], [2.038363806966415, 49.19240704737498], [2.041633084239522, 49.194025401943996], [2.056298990835978, 49.19933253619818], [2.058763928811099, 49.19996679439388], [2.066842351201006, 49.20286085512456], [2.072610422979904, 49.20608904145239], [2.074911854840503, 49.2086825974711], [2.077443774199375, 49.209676607977194], [2.080882733781487, 49.21044732058307], [2.080885091998517, 49.209773929399496], [2.083038632582717, 49.207891480187726], [2.087137228519729, 49.20736157086035], [2.089589471788674, 49.206030603026896], [2.091539850615211, 49.20436044469346], [2.091763859444134, 49.20051046078612], [2.092554538366664, 49.19563797159929], [2.093319605330557, 49.19339330873835], [2.096207813481966, 49.19132372269302], [2.095870392956305, 49.19006515747348], [2.096879892706375, 49.189757147696625], [2.101455983110403, 49.18944431659683], [2.110307896315312, 49.187763077990994], [2.111048295998629, 49.186982735539786], [2.113336904057638, 49.18654376178135], [2.114337906353577, 49.186641934200566], [2.117897197951979, 49.18745655588539], [2.121321242961235, 49.188406746506786], [2.122073069971552, 49.18872241474506], [2.125436094707557, 49.19093885897184], [2.132740260012667, 49.19137916940763], [2.135147083896186, 49.188299109823774], [2.137280066811563, 49.180482398686934], [2.137214495724094, 49.17980850490271], [2.13790196748389, 49.17904559774298], [2.140762187499047, 49.178918584522386], [2.141182789098997, 49.181341953811916], [2.140688513499184, 49.18533939804555], [2.1452965447148, 49.18604110650761], [2.152089738205949, 49.183561768914814], [2.152383154005789, 49.183819176285795], [2.155721334860415, 49.18248858690476], [2.164570733641903, 49.17967139320751], [2.164914553113736, 49.179357296178694], [2.165122106511589, 49.17881837888723], [2.164986649751793, 49.17845870072582], [2.157743439339026, 49.17259918558599], [2.162031785553346, 49.16868024089206], [2.162912157412535, 49.16740334076571], [2.162472043703076, 49.16714763158385], [2.163931489341622, 49.16609413086406], [2.164274336828493, 49.16600480202056], [2.167792580491651, 49.166431969162076], [2.175927689416838, 49.16960151542588], [2.177833519217356, 49.17194141626579], [2.17724721518527, 49.174233711098395], [2.174946208634322, 49.17500378083456], [2.176092739790298, 49.17603210511916], [2.181952799107559, 49.173675958449714], [2.187368324705129, 49.17322351396026], [2.193263486745546, 49.173400711156035], [2.203786142659832, 49.17415675101728], [2.206289499473805, 49.17464709846414], [2.213103991171729, 49.176516974872996], [2.216075177130751, 49.17890994974269], [2.218645881572126, 49.18068527088098], [2.219790958165723, 49.179782816917395], [2.220271537113421, 49.17919882433455], [2.220478513529675, 49.17865981169942], [2.223914666020093, 49.17394649081366], [2.231460121103116, 49.170215089174924], [2.235409518270254, 49.167035850116875], [2.232949562049753, 49.165473754959685], [2.227706143769907, 49.16320327473407], [2.222851352981508, 49.15964125464803], [2.217722877513007, 49.15563757900906], [2.216354665810456, 49.15437801734082], [2.217521073986123, 49.15366092067436], [2.219852548366139, 49.15276435319634], [2.223004977789504, 49.15186944566514], [2.229580694623436, 49.15129132287349], [2.231991733562129, 49.151293243070405], [2.241579846645345, 49.15157072032995], [2.241852506877208, 49.15251470581792], [2.245603756907523, 49.15339242682564], [2.24994665238854, 49.15346362073421], [2.252481360722899, 49.152881479905346], [2.258026104357374, 49.155266948274715], [2.259835588301498, 49.156416354402154], [2.262209120085148, 49.158297463046516], [2.266003564970822, 49.15666450844228], [2.269545378848687, 49.15630662866976], [2.270230142186366, 49.15693655754948], [2.271462426854212, 49.15743133478299], [2.275639195766884, 49.15862447087297], [2.286406160159514, 49.159917838892895], [2.2862682227561733, 49.16027755982778], [2.286651979075863, 49.16635592328175], [2.288806483960534, 49.17071037741772], [2.2957119062354048, 49.17322884524951], [2.299422953351624, 49.17583535834244], [2.299765539147143, 49.1772731854557], [2.299831678846354, 49.1802917832025], [2.3000354715167752, 49.18154987950567], [2.300445606624441, 49.18276331945223], [2.301199861528492, 49.183887047525026], [2.304491035585571, 49.18392857258516], [2.30687576819463, 49.184163490850715], [2.30852588591032, 49.18538582570099], [2.309971388937101, 49.18519182311541], [2.310587309739558, 49.18487707762948], [2.310930440140444, 49.186404768173134], [2.318757169775381, 49.18523823433531], [2.321772911616363, 49.184429902923966], [2.32156796042575, 49.1841608249203], [2.322262675365499, 49.18085973633791], [2.34626253429462, 49.16181585433701], [2.352357915893295, 49.15556143524614], [2.3576983008356, 49.14850694519109], [2.359299852747084, 49.1473439500781], [2.370937678429021, 49.159173609643325], [2.371252751326479, 49.159305623702785], [2.373160719283445, 49.1593317562613], [2.374031887294624, 49.1591645624539], [2.383003341928025, 49.15591300483808], [2.389368304455063, 49.1065338476737], [2.391284726289088, 49.14935040839237], [2.39513423157857, 49.14975282928344], [2.3998607761522193, 49.150648610949446], [2.415354480539764, 49.15176241395538], [2.432826963987786, 49.148102177084674], [2.440903978628781, 49.14580411256264], [2.435401802091311, 49.1339390401983], [2.439545429277035, 49.13418318733546], [2.442386812507328, 49.13460727943461], [2.447058385061109, 49.13559177873564], [2.453295719423779, 49.13832568132753], [2.453776426784377, 49.13861198060626], [2.453601705656181, 49.138807191824014], [2.454324501921219, 49.13926811434213], [2.45823091553417, 49.14114690208868], [2.460681526716542, 49.14006190531988], [2.462486436140128, 49.13912502480333], [2.461302566423911, 49.13661033564837], [2.461506886215731, 49.13593692199041], [2.475749328292375, 49.13106911647333], [2.477597666264542, 49.13106636593209], [2.477460176379238, 49.13084189113665], [2.481707071371559, 49.12691946353657], [2.499232118962853, 49.1222697472609], [2.502096193380512, 49.11888747545116], [2.49158738825412, 49.11130889016783], [2.489929557393243, 49.106359373929536], [2.504032476214566, 49.10377941699439], [2.505195036842471, 49.10364304250646], [2.507658202455457, 49.10368462132817], [2.520051820464353, 49.10200389729827], [2.531058595768549, 49.099597191310124], [2.532994448247441, 49.11924516207648], [2.538616568664839, 49.12166209195825], [2.540808712231494, 49.122242313786494], [2.541973137907616, 49.122240458805685], [2.542656940343452, 49.12205964159674], [2.543475814276615, 49.121473833959406], [2.543747291748411, 49.12097945334663], [2.543881022504587, 49.120259771633854], [2.543673438169058, 49.119721315013656], [2.542987578014889, 49.11936266263256], [2.541960810801747, 49.119140273036805], [2.539633296580424, 49.119009109020396], [2.538535554741946, 49.11860659556171], [2.5379859831577303, 49.11802365813324], [2.537778469020513, 49.11748519015767], [2.539692567041721, 49.116808302787135], [2.542975759912367, 49.11635328106501], [2.547150593969974, 49.11621082562427], [2.547641311135663, 49.11899543150314], [2.547441293129697, 49.12025343709597], [2.5477858996788, 49.120792401915445], [2.548967524795592, 49.121958489362854], [2.552604206467529, 49.12406339703002], [2.554932366550416, 49.12414935545013], [2.557018849556176, 49.12382168849185], [2.557140970355757, 49.121714625724145], [2.556155065446946, 49.118627938324074], [2.554442915059725, 49.1159080817859], [2.55201490542728, 49.11394708174554], [2.551529367459702, 49.11260019175068], [2.5516643555728438, 49.11219519235352], [2.552617736592075, 49.11107034720173], [2.555073996349331, 49.10908880498606], [2.555757793366885, 49.10904278001565], [2.557589220998047, 49.105310976680435], [2.558130860289426, 49.10378175739545], [2.558156406797649, 49.101908969936424], [2.557839496697433, 49.099730115436], [2.558175509896552, 49.09838175600357], [2.559471867863822, 49.09761505771575], [2.571081015879054, 49.09520959130482], [2.573154509911697, 49.094309744666404], [2.572466458375747, 49.09340277142212], [2.572465936643884, 49.093133928377235], [2.573834606229447, 49.093355478676614], [2.574654463485853, 49.093219015971265], [2.576773821984558, 49.09272118117381], [2.578685865418398, 49.09195284211966], [2.579906144729438, 49.0897038769077], [2.578464441854639, 49.08853894311046], [2.57971786286995, 49.08590795416967], [2.582835698000931, 49.081726114254124], [2.583008604161083, 49.08065043838731], [2.583303814120438, 49.08070630993654], [2.584733459837687, 49.08011512425871], [2.586557881384846, 49.079791426906986], [2.590528392673529, 49.07965396159913], [2.590598813327991, 49.07992304585781], [2.594929043195068, 49.08139702703953], [2.5960243975913597, 49.0817091192789], [2.596639358190874, 49.081662630636664], [2.598630366318649, 49.082804012872465], [2.603296504767269, 49.08588017590401], [2.607212181823519, 49.0890615936403], [2.608188372231614, 49.09013744764155], [2.610325438125285, 49.09305309045343], [2.610334531202718, 49.094940397032175], [2.618749519067133, 49.094740792343465], [2.619363525531608, 49.094469401961156], [2.620530604159276, 49.09514114465725], [2.62704663135949, 49.10083142764603], [2.631759579539446, 49.106264959944404], [2.633277099126323, 49.10837263509713], [2.634302884425047, 49.108370363720404], [2.63655364491212, 49.10710673244202], [2.6403799844436913, 49.103553223852195], [2.645969761647612, 49.10015494105567], [2.650214675215634, 49.10054859182419], [2.658239453169009, 49.097546111677936], [2.667693496069663, 49.0924952360602], [2.67790944880482, 49.086141896070785], [2.678524244050843, 49.085455703201454], [2.684801338099397, 49.08107014948275], [2.692195415641706, 49.07209878290797], [2.691158648921952, 49.070618824222755], [2.689773384432125, 49.06792680044628], [2.689906902864157, 49.06756748485688], [2.690995242686318, 49.0665758272298], [2.696351631291035, 49.06509958646265], [2.703084745686811, 49.06473337258461], [2.706179465005917, 49.06530744516173], [2.715316518839677, 49.071218863926575], [2.720486228010274, 49.074886920185364], [2.720693056303956, 49.07511038842399], [2.720972902052352, 49.07600838246914], [2.720232901248996, 49.07744888503977], [2.7199650183106323, 49.07839235856969], [2.720104942524888, 49.07884135518857], [2.720589071620381, 49.079424224617455], [2.724358603980736, 49.080454348952955], [2.724353783918386, 49.079815949769404], [2.729307656977005, 49.07476774411045], [2.731608667305832, 49.07152519613576], [2.732416501435415, 49.06985638393578], [2.7350099332564053, 49.06045251665067], [2.760834754176539, 49.06300544039254], [2.769626037900153, 49.06765593355351], [2.775818787924691, 49.07068787657977], [2.7802721121307252, 49.071615026287034], [2.787073783911977, 49.07527418832183], [2.787075598526116, 49.07549897657354], [2.785715727323252, 49.07635793570532], [2.783875585630323, 49.0771728575587], [2.779373046177838, 49.07835783387731], [2.774123498259577, 49.08036304998383], [2.7683329826623, 49.083259832689144], [2.770593932683987, 49.0836561908642], [2.772646080177827, 49.083603444100724], [2.778794415216922, 49.08277242675206], [2.788864522614811, 49.08286965910479], [2.789003232045312, 49.08309378687943], [2.78893943936353, 49.0836331547676], [2.788536429909293, 49.084443464079094], [2.787113349291003, 49.08597666911659], [2.783989343574372, 49.08863881466793], [2.791731796468083, 49.09013693622857], [2.794926586117342, 49.091652960240346], [2.809052850505203, 49.09753760539617], [2.809258239144329, 49.097536133127804], [2.814749500913115, 49.091979150582596], [2.819850264792854, 49.087367188284894], [2.822098613949588, 49.086414762099096], [2.827347808501511, 49.08455132527349], [2.829530823336189, 49.08382344693074], [2.833626504104517, 49.082952531440135], [2.834651917886291, 49.08290359483363], [2.838157514917431, 49.083338150670905], [2.839462021652585, 49.08378137346266], [2.844945110709704, 49.08488191377207], [2.855742484402942, 49.07026011103562], [2.856082499785197, 49.07003393466598], [2.859026555743457, 49.07038089670244], [2.866267865713643, 49.07069398191056], [2.874864640083245, 49.07305089778596], [2.880072029295075, 49.074840098835786], [2.8830747436616333, 49.07607127967779], [2.885546472251984, 49.078578652007], [2.887615416314578, 49.07946805007555], [2.894948530364222, 49.077063202005434], [2.895376285751489, 49.07738097012923], [2.8968765033548998, 49.07937744723244], [2.8994918562751, 49.08377462071547], [2.901507882228152, 49.08537314344544], [2.906464404999735, 49.083237173318466], [2.917450691214286, 49.07981593007686], [2.923408059285361, 49.0794323470372], [2.929397388329237, 49.07963652093232], [2.931220705012413, 49.08055739120962], [2.934774304665712, 49.08033998357658], [2.938105289839478, 49.07853718370313], [2.9378937827005562, 49.0779993920905], [2.940481624875066, 49.0771321367115], [2.941442240270374, 49.07730704083263], [2.94241151961885, 49.07831364120247], [2.944584253375572, 49.081604590570365], [2.9448730076911, 49.082950722565045], [2.944813382573614, 49.08366999934947], [2.944478697623972, 49.08431002550483], [2.944416520419733, 49.08484947421454], [2.944772556904112, 49.08601580773697], [2.9458230982897082, 49.08807708460737], [2.952777845913551, 49.08614509661618], [2.956530703219955, 49.08536247257818], [2.970473592918939, 49.08897400169044], [2.9714318898040313, 49.088968838468254], [2.970730406725954, 49.087534564253524], [2.970942472349372, 49.085309275638274], [2.973177504380649, 49.080736818579076], [2.973905324353198, 49.078747197170536], [2.974562001485486, 49.07658671668412], [2.974609393826399, 49.07483430938709], [2.987996419090343, 49.072371853393314], [2.988133274417306, 49.07237186698378], [2.988205736399943, 49.07268567376061], [2.988032704646126, 49.07525629255322], [2.987703977425113, 49.076337021485415], [2.987852951009928, 49.077234374030915], [2.988848695547018, 49.08028433287398], [2.991236555799942, 49.084010529457444], [2.99333896575986, 49.08525326034383], [2.998202566308823, 49.08747337671501], [3.001887827809002, 49.088662918754814], [3.003950891148115, 49.088737496053994], [3.004383400856272, 49.087677411894425], [3.006047909646361, 49.08645003177047], [3.008515542335193, 49.09150656131351], [3.0155578961164, 49.091233422048056], [3.029524675403112, 49.085932257521314], [3.031572368113765, 49.08560625289738], [3.032781928013719, 49.08906756622174], [3.03312417530109, 49.08906567416511], [3.039660701846532, 49.086726866948254], [3.04835577703769, 49.0863163793526], [3.0556567236042, 49.08804673748802], [3.057953584222672, 49.08780828293327], [3.066070953443055, 49.085211911434016], [3.066017513928712, 49.08624593817177], [3.064342308027294, 49.08873652796721], [3.06269348199221, 49.09053205198453], [3.058992637262631, 49.09263247780538], [3.057355340018481, 49.09399004422695], [3.056229537780318, 49.09616284959491], [3.05591632390162, 49.097737351670304], [3.056026326305765, 49.10070168669049], [3.056316565202643, 49.10191355733038], [3.061099263647705, 49.10507430033909], [3.065461901480414, 49.11132440472984], [3.069679270608455, 49.116735536536915], [3.071332165051532, 49.117466435654855], [3.071880183350189, 49.1175533221721], [3.073211092261369, 49.1172099623133], [3.076229507203222, 49.11501607791759], [3.078780387846411, 49.113566878236576], [3.082429964032613, 49.11237941305132], [3.090348024043238, 49.11100438837842], [3.101945087526206, 49.10866553092374], [3.112967865980438, 49.107840191050904], [3.114519342157877, 49.107149106682016], [3.115731672241329, 49.107304391345515], [3.123410033096801, 49.106604442776785], [3.123697092349844, 49.106957498288324], [3.128897723711474, 49.10671287365391], [3.13542848974344, 49.10558900092663], [3.141062013725083, 49.10396142533922], [3.143771357341303, 49.102235514321656], [3.149528689818946, 49.10054804626653], [3.152769735208814, 49.10005395394916], [3.158523215885834, 49.09952052348057], [3.16310909166103, 49.09957822803817], [3.164411813315073, 49.09974908455783], [3.165230337654597, 49.099653555328565], [3.161897760538122, 49.09315295232496], [3.160563217401365, 49.091049892312434], [3.156868739236467, 49.08671738140469], [3.158617005880282, 49.08049579481462], [3.163581307972858, 49.07868727487276], [3.168670325261773, 49.07619412734975], [3.176633914747797, 49.06957849528211], [3.180121394872446, 49.0654111716702], [3.181844795038826, 49.06207363097592], [3.181825493665108, 49.06086070497492], [3.180799945453969, 49.056725246433054], [3.180859474747383, 49.05614070561958], [3.180577138827619, 49.055603434887], [3.181763555561881, 49.05294373594442], [3.182165068832243, 49.052356876117], [3.189962252951226, 49.05257138835231], [3.190756917331472, 49.051037960567726], [3.19090574204521, 49.04761281555326], [3.190616559459862, 49.04671680826537], [3.190202644715089, 49.04649447157626], [3.183707076755577, 49.04508120280705], [3.181201109613966, 49.04231287812749], [3.179854240874247, 49.037290340081405], [3.176533029656682, 49.030276306390135], [3.175760397018716, 49.02946098564273], [3.174067174463159, 49.028540894145515], [3.171933420731614, 49.02918328474308], [3.168088821287292, 49.028095325800635], [3.163664807262521, 49.02663217641392], [3.162487185468441, 49.025651869679315], [3.16102929171019, 49.024224133703314], [3.161874058099979, 49.021522747676826], [3.162338965294111, 49.02066519597541], [3.163010722432934, 49.019942239094966], [3.164795045414459, 49.02042356552805], [3.1716785230799562, 49.014125889686824], [3.183651670855833, 49.01125053274095], [3.189590951244657, 49.0089753255707], [3.195851831472066, 49.00549256884932], [3.196044736973492, 49.004817853756514], [3.198309091422146, 49.005430085996394], [3.198997948484385, 49.0057850290195], [3.199670884694577, 49.00519583459921], [3.207164771772392, 49.00068712164542], [3.208511694097662, 48.99883152717153], [3.20876891939501, 48.997886014075426], [3.209314063913049, 48.99378377435688], [3.215440839940333, 48.992660606753944], [3.223672035184652, 48.990038727830985], [3.224303695355136, 48.98791270403759], [3.224840852587758, 48.98732451058299], [3.226092169598182, 48.98789945579459], [3.229174161316664, 48.98841515359854], [3.231212785128233, 48.97675284910678], [3.236461987282692, 48.977544725262014], [3.244197398485848, 48.976835025426894], [3.249340975874024, 48.97435966288707], [3.251096015547074, 48.973178012313696], [3.252362325854711, 48.97150548488283], [3.253616854441953, 48.96911447958972], [3.254463132772537, 48.96677108268591], [3.255102584227691, 48.96376357859166], [3.255210267838715, 48.96205209977218], [3.25714257744932, 48.95706258100211], [3.2572114351874912, 48.952487075837745], [3.258441253127004, 48.95103034821878], [3.261212175460174, 48.949517202301166], [3.262968265554206, 48.94903226472821], [3.262918000987463, 48.94881205874589], [3.2622782227770513, 48.9483800215396], [3.261917189918176, 48.947177613248364], [3.260655848598628, 48.9450311496875], [3.260172724734463, 48.94479030010069], [3.264874403315253, 48.93914436764619], [3.267847397739333, 48.938330226805284], [3.2737719569167583, 48.938907654179175], [3.284526511757878, 48.940530948463426], [3.295688504323757, 48.94478709206264], [3.304753582454241, 48.94876696624783], [3.309108520935207, 48.94157796382], [3.311033861691, 48.938011915753385], [3.312723157027109, 48.934277378775256], [3.312874803290789, 48.9336196206705], [3.313036276757539, 48.92915164254215], [3.312883172099563, 48.92616017210456], [3.313077233236627, 48.921215462323], [3.313769760834584, 48.92110214069629], [3.315327061828095, 48.92047755238826], [3.326699178511224, 48.91283738167595], [3.330294003482512, 48.908707002959346], [3.344121555542807, 48.915424399688035], [3.3535717935991523, 48.915243105487995], [3.360535905438951, 48.918399510138684], [3.36634585957629, 48.922834212294276], [3.367401358808225, 48.92736961892258], [3.36705966082186, 48.928071199171804], [3.367151713694452, 48.928351490007636], [3.367477685705575, 48.928321714162514], [3.367472961907862, 48.92803935725969], [3.372205568843475, 48.92168486300936], [3.375504541820247, 48.90722397938011], [3.371030999947087, 48.9033619751126], [3.369007685601208, 48.89388784267284], [3.371343381891012, 48.89207932239483], [3.382811849140077, 48.88874366317077], [3.381430493157589, 48.88131259076185], [3.380455229118017, 48.87417513956177], [3.381076285224036, 48.87278371828621], [3.382685294596448, 48.87170929514069], [3.389742059565488, 48.87120808823325], [3.393326383098576, 48.87322702707481], [3.396664366840953, 48.87562438545049], [3.400754532406561, 48.87597408656771], [3.4058488876004662, 48.875883179746324], [3.405583607266972, 48.87228140866214], [3.404900361303263, 48.870849305441816], [3.398896647226793, 48.86641871601601], [3.401547905314557, 48.86552311607649], [3.403220841926283, 48.86469011194024], [3.420973530914235, 48.86394999286298], [3.436169331946029, 48.859878268849215], [3.442791412994167, 48.86043761648885], [3.449574445045652, 48.85799222074503], [3.452235173122167, 48.85629504222796], [3.451707676797841, 48.85534553685841], [3.445770993464591, 48.84706149963301], [3.445362133428643, 48.843227318169376], [3.447681326054025, 48.84276539124629], [3.461905863476263, 48.837795469893315], [3.462753573461165, 48.83721391026651], [3.462804510677076, 48.837426859600946], [3.462515960213532, 48.8379190216476], [3.462655329063503, 48.839806219720366], [3.469109357646495, 48.848738577463145], [3.470466582499343, 48.8509995908902], [3.485183372194707, 48.8519103095659], [3.486854174632986, 48.8492873344919], [3.48957425065192, 48.839868156724464], [3.488456893192224, 48.831276688656054], [3.484968448663741, 48.8250097488729], [3.485649767462931, 48.8200676695204], [3.486369521465548, 48.8169241488876], [3.4872017702391442, 48.81519037154211], [3.483097905131964, 48.81364751326409], [3.48092871405028, 48.81218578415147], [3.4790610976821412, 48.813893096209064], [3.4779368885941198, 48.814110746535825], [3.474204074190505, 48.818290590556515], [3.474365662351537, 48.8188367651896], [3.474840398044131, 48.819371803180665], [3.470381562708658, 48.8208976636916], [3.46276269650486, 48.818138456804725], [3.454412181494484, 48.813840024327526], [3.451499878278679, 48.81311897022172], [3.44265621824516, 48.812118663630024], [3.435615333544101, 48.811875613327324], [3.432166496861304, 48.812257770937734], [3.419271606035608, 48.81642818665349], [3.416617646714704, 48.81782878144234], [3.40416496463084, 48.80927626877178], [3.410477888420274, 48.80394306174327], [3.411140424518429, 48.8037033497088], [3.4152090219831033, 48.803407782040765], [3.42540209015586, 48.80116486917516], [3.430448596503113, 48.80177456856848], [3.431953243021198, 48.80212979133433], [3.435893448343619, 48.80344206995069], [3.439599043803823, 48.80258038654941], [3.441289582497735, 48.80091930017646], [3.442548105584842, 48.79488518977198], [3.442745335408552, 48.788771326518784], [3.442609261297413, 48.78813416339079], [3.4429070953147143, 48.786180482871316], [3.442160723332969, 48.78435388354638], [3.433869705538184, 48.78079029899412], [3.42944781145829, 48.779437394608244], [3.428211152964617, 48.77948500420177], [3.427563824832289, 48.77970677500548], [3.425682537190969, 48.78147720553889], [3.4222674476041473, 48.78293105101242], [3.418157425239015, 48.78413719950413], [3.409978575816222, 48.783723041296774], [3.409422809689177, 48.78383557201928], [3.408195780422094, 48.7823458804873], [3.408401156665255, 48.78111211119467], [3.408242184702729, 48.77992725355023], [3.407380497086681, 48.77780492438071], [3.402590565594843, 48.76972207565173], [3.400840903078306, 48.767256400850115], [3.398657955436997, 48.75559928198552], [3.399113141564564, 48.754999651964894], [3.400051216534954, 48.755289721827786], [3.402943382450685, 48.75444174481643], [3.412961338094533, 48.75390205084498], [3.42566951875466, 48.75568953623999], [3.435810346977949, 48.75357171137704], [3.43659053639097, 48.75317761524154], [3.436286377372976, 48.75226401955927], [3.436956569984682, 48.74474241319227], [3.439159632693516, 48.74036222366024], [3.440606129229001, 48.73870194682349], [3.44237665961819, 48.73758190602534], [3.444695976408552, 48.73672422704318], [3.451815198779946, 48.73602831779368], [3.465024262836441, 48.737898777870726], [3.467391453240857, 48.73863156348331], [3.469726171993308, 48.737863280395445], [3.467467022278121, 48.735224220885414], [3.466893819337418, 48.73133731012027], [3.466743129398115, 48.72877630337814], [3.467157728616321, 48.72283927589482], [3.467077882969539, 48.721663117130625], [3.466161936915055, 48.714534091559166], [3.464308279198275, 48.707378088033074], [3.466808722925235, 48.70492720917507], [3.4692478400447593, 48.703088144555885], [3.472813271343747, 48.70184900422773], [3.476969053517772, 48.69935534710221], [3.477772629139833, 48.69792558494986], [3.47627998547524, 48.69754928269869], [3.471763442222627, 48.69152552837103], [3.465831800083378, 48.68619250911084], [3.462679014103557, 48.68544922753363], [3.455423620695103, 48.68265011772279], [3.448601448364572, 48.67785120003462], [3.446831224853806, 48.67509915238519], [3.444007154571354, 48.673129951157094], [3.442691260176015, 48.67250341014517], [3.443316805119228, 48.67195869958904], [3.443817381655, 48.67125254428393], [3.446181304929037, 48.6668974230096], [3.44061617596667, 48.66365029472754], [3.440848937962538, 48.66307286352133], [3.442015059819828, 48.66239662851224], [3.442568100016932, 48.66228304004844], [3.44919569110172, 48.65927374522692], [3.457258946100309, 48.65533119487151], [3.460431735459521, 48.65300907834938], [3.456190797767568, 48.64493800160113], [3.455677095047254, 48.64143910445248], [3.453580462720506, 48.63663115351226], [3.453297980792156, 48.636139298707704], [3.451096034944055, 48.634649976025436], [3.453116217559373, 48.633955984320295], [3.461843964974497, 48.634898436047166], [3.472622391950873, 48.6365580347156], [3.476573378332905, 48.637329365255624], [3.485103006006336, 48.64153521647296], [3.485959198285585, 48.64223693963559], [3.48819415876491, 48.644795887866316], [3.495515728897584, 48.6454897662143], [3.503184252987448, 48.64568250607798], [3.505270949429034, 48.645015239109945], [3.510434082731254, 48.644136904823114], [3.517498400145996, 48.643365240304234], [3.533239157740262, 48.646507074405896], [3.5328391911316572, 48.64562105155237], [3.5288700214598903, 48.642793594723976], [3.522230825421071, 48.638535819630036], [3.520817665174046, 48.6378259951014], [3.5182915782067523, 48.63765800842339], [3.518409945914831, 48.63631722160039], [3.519064367834498, 48.633470112330095], [3.522271857392224, 48.63258356455047], [3.52567507738048, 48.6325497059121], [3.529125611678456, 48.632766504669064], [3.534822686526187, 48.631971977770185], [3.540491301185718, 48.63040911821547], [3.544603879861267, 48.62878924376352], [3.547839692470334, 48.626787365181684], [3.554761799323597, 48.621830435555744], [3.555445367842173, 48.6209106328168], [3.555613758785007, 48.620285576056155], [3.550832909739722, 48.61774413116045], [3.545812402796253, 48.61598348176486], [3.538821394565017, 48.61214374155708], [3.534618142122431, 48.61035282901075], [3.503375661623198, 48.60462315315539], [3.503503754589771, 48.6042618982418], [3.504165183199048, 48.603508887185214], [3.507770368320342, 48.599879227553345], [3.510578713667234, 48.59761117225112], [3.514016528600822, 48.593180527133164], [3.515863913658221, 48.58978215089348], [3.506585176887138, 48.589379427270025], [3.497505711492652, 48.58995475250354], [3.491010753144396, 48.58722281791302], [3.487756453708648, 48.58356784515567], [3.485274703417289, 48.58042686834431], [3.473382714190322, 48.576873863924426], [3.469017239384725, 48.573050056535806], [3.465521572790446, 48.570484758127044], [3.472296134883944, 48.56491317677625], [3.476936986207372, 48.55540605151196], [3.479770042872202, 48.550623337168744], [3.479600767022518, 48.54470931994303], [3.481750724291408, 48.541453154630105], [3.477090301733287, 48.54158079318113], [3.4756567328616192, 48.54140122306167], [3.473590957931793, 48.54051978519244], [3.469689182089071, 48.53852470129303], [3.469136920496923, 48.537967342619524], [3.463920883502268, 48.5340575367895], [3.459184862613696, 48.53074026568639], [3.456947174373788, 48.5296343013749], [3.4507018022812, 48.5286667990301], [3.449480600245807, 48.530457998647755], [3.448101733234727, 48.52981730205412], [3.446747741840105, 48.52891291728579], [3.438227851902585, 48.528333810560696], [3.435153475007155, 48.528829980068714], [3.432611013398123, 48.52961739966904], [3.4301202896338863, 48.53127267455072], [3.42344776953071, 48.533559074884614], [3.41548710883731, 48.532334944650984], [3.414239107151204, 48.53346827308611], [3.405396732567803, 48.52801490292368], [3.420687265839886, 48.51734753481927], [3.423914294772555, 48.51436021418381], [3.430144578765275, 48.50236236559511], [3.433376733249279, 48.493287982484844], [3.433433883647076, 48.492177654824474], [3.434638738307846, 48.49025168258217], [3.420354597162878, 48.49154724893413], [3.417069045350757, 48.49025529363933], [3.415226669667496, 48.48815035911571], [3.412172471453859, 48.48732435633567], [3.403508857382822, 48.48595520446669], [3.396407024084616, 48.482931225488244], [3.388358481444744, 48.480409204058304], [3.384090868640628, 48.478028301653225], [3.38594753446, 48.47787834362037], [3.388494314271247, 48.47718812918754], [3.393149643681093, 48.47007209762206], [3.396439275848441, 48.46372869094791], [3.401046580155309, 48.458767948756595], [3.405708524863503, 48.45328952824054], [3.406247344796445, 48.45283606791072], [3.40648375193441, 48.452452013320496], [3.406894198560617, 48.44974449625209], [3.406789120863873, 48.44857442575344], [3.4048841374713232, 48.44389290917], [3.40356672053998, 48.43873883468228], [3.401121921897516, 48.43675530556473], [3.39899569190829, 48.43628476090404], [3.395145138679572, 48.436815842317365], [3.394561432110056, 48.436143948722645], [3.397034929447667, 48.43480511886995], [3.396988521493739, 48.43456776688342], [3.39536239746309, 48.43065162156391], [3.392138085238996, 48.42519782880008], [3.391958501583378, 48.42431765975411], [3.393285456581294, 48.42408295639231], [3.394780104800855, 48.42422103739886], [3.400975445397588, 48.423939359368084], [3.404760769229023, 48.42349287044546], [3.407748212992982, 48.42282858933919], [3.4118359218741072, 48.42131563855661], [3.411547943674732, 48.419923965471654], [3.410338032648072, 48.41868032832947], [3.408693078363557, 48.41762170843934], [3.404826208917381, 48.416721812133105], [3.402411602850576, 48.417671973330755], [3.401751736773806, 48.41621495961355], [3.402162395887253, 48.41530670871198], [3.402497853403637, 48.415168822192896], [3.409028672788871, 48.41420183494237], [3.413552944130866, 48.41416095637707], [3.413623273922728, 48.41433974422505], [3.415573568620868, 48.415558268676094], [3.420762539820354, 48.416657356763864], [3.421504093004039, 48.41660524256867], [3.421972933496137, 48.416420944116545], [3.4221035695897353, 48.4161955636121], [3.422079296488612, 48.413341016515645], [3.420970614604204, 48.41040664816049], [3.420677857745812, 48.409284902298936], [3.420874326169795, 48.409013405301856], [3.419757747488143, 48.40727020665913], [3.4181889023179473, 48.40386779936543], [3.416603821683492, 48.399072725294864], [3.414788822463184, 48.39026854999573], [3.411478476226268, 48.39033316667383], [3.402032816849439, 48.38970610866454], [3.397604707612198, 48.39164725451979], [3.394565151403849, 48.393608809225206], [3.391592117556152, 48.3960073047326], [3.3833174045242362, 48.399704111030054], [3.379449811858255, 48.39902200361159], [3.377475457158693, 48.39838953785638], [3.375401057671634, 48.39868671752446], [3.3750520786382, 48.396734632184554], [3.373482225482663, 48.39349895988309], [3.367213028666657, 48.394317453732675], [3.362784782448327, 48.3825767871233], [3.362607187800485, 48.37699478954528], [3.363059185456087, 48.375956966987715], [3.365720149153585, 48.37584712143394], [3.365160630192652, 48.372282482286195], [3.356605174752673, 48.370941905779894], [3.356408516900455, 48.371213304004755], [3.355913893267632, 48.37634301198695], [3.356562887687407, 48.37844992435729], [3.352989202186535, 48.3786153667161], [3.350964902215543, 48.37863307383975], [3.348418713318873, 48.37326046262154], [3.337483596829866, 48.373083377951446], [3.336540315922298, 48.37318142712144], [3.3344861196000473, 48.371862902034486], [3.331005996914639, 48.37310254600524], [3.330109943175337, 48.37212078709896], [3.320011738026405, 48.375617674267666], [3.317068865950932, 48.37646768826579], [3.312063166700705, 48.377224112287294], [3.309321621761203, 48.37698028698419], [3.309486723969424, 48.375891238396896], [3.309953793989042, 48.37494354959212], [3.310678900670133, 48.37401767424906], [3.310899547515052, 48.3729653627047], [3.305158635552329, 48.37287681967105], [3.299334269046772, 48.3754422803949], [3.295247443809388, 48.377049469588854], [3.282829883778889, 48.38156915874444], [3.282392164602431, 48.377519839774806], [3.276646891870564, 48.377199606699094], [3.273151733242015, 48.377172625454214], [3.272181545182486, 48.37889324867134], [3.270395206357731, 48.37818119117746], [3.267623312880799, 48.376721001279996], [3.264776782259218, 48.374621237372374], [3.254386262297955, 48.36502005160056], [3.253911940949558, 48.364933799031036], [3.251685446578171, 48.364915157026935], [3.232849124600888, 48.370332895504745], [3.226025007454056, 48.370078900614985], [3.214895597659826, 48.36767675135189], [3.214172387588383, 48.36747023307276], [3.213601904644363, 48.366967438270684], [3.209665843231891, 48.36555836024394], [3.205808583631303, 48.36491194904969], [3.204447574871326, 48.36420265180103], [3.201850550002267, 48.364015470519234], [3.198347033617226, 48.36654051145422], [3.191902794197647, 48.3680850869017], [3.186231231598984, 48.36830106305763], [3.184946555144015, 48.36813033542602], [3.182534129483375, 48.37331715134179], [3.181872408813525, 48.3740864964543], [3.180272502165224, 48.37535659067133], [3.179671459867881, 48.37572097404767], [3.174836420444186, 48.37719378243594], [3.171807544146094, 48.37767414150398], [3.169107772837202, 48.37342163937832], [3.167733513997738, 48.37185814622567], [3.164113004768283, 48.371336001240834], [3.1524290205684222, 48.37143617501327], [3.149256269462736, 48.36970652487944], [3.147410734591492, 48.36788157714458], [3.14488318487757, 48.3683984139095], [3.141319075430282, 48.37060334272945], [3.139864713127896, 48.372599662435015], [3.138425466831908, 48.372171304602794], [3.132463658521117, 48.36945927873079], [3.12845030031724, 48.368202340654875], [3.125694038113584, 48.36801996129815], [3.122464351742041, 48.368626130899635], [3.118255559667571, 48.36678785281526], [3.109730209002628, 48.35697686223136], [3.109020119880124, 48.35566486799148], [3.107290275756352, 48.353380328289454], [3.105466503834771, 48.35129917427134], [3.103415343992539, 48.349510589008545], [3.1015380716537, 48.350331835619016], [3.099600853032197, 48.35160384932316], [3.099406150898821, 48.35214382939744], [3.099622471650698, 48.35304154205373], [3.100104293595582, 48.35371320675587], [3.087841104647601, 48.358744110392905], [3.071915600335573, 48.35898156860354], [3.070151501135413, 48.358227786826156], [3.067716935346246, 48.35779375344006], [3.062585284749021, 48.35742950174769], [3.060548216239474, 48.3574422562522], [3.059472563206067, 48.35767402139451], [3.058332401455334, 48.35817571539786], [3.054263266451775, 48.36049254802768], [3.050604522220211, 48.360334922340805], [3.049522942778149, 48.36011675512152], [3.049244208915181, 48.359444804049865], [3.048349352133347, 48.35819100605075], [3.042560778813888, 48.35166236539918], [3.038805238676076, 48.34566088494031], [3.037490238158603, 48.34315114936519], [3.0365510358338, 48.33942488278878], [3.036277235373083, 48.33767954417468], [3.037571499647933, 48.33031595089154], [3.036574116485578, 48.32602824842645], [3.032608593353325, 48.32325199704647], [3.027605564981934, 48.31945287815799], [3.021017167365649, 48.312767532707824], [3.015898309686554, 48.30789724140426], [3.015689089460483, 48.30731334923752], [3.014777081752486, 48.30622210221763], [3.015476709740117, 48.305140544510785], [3.019400458594804, 48.30318036525837], [3.022825815130045, 48.30318245338647], [3.02744313623055, 48.302862969882035], [3.027255744577942, 48.30036807096593], [3.022739273182386, 48.296392223547784], [3.020414850704682, 48.29343972999329], [3.028298776301811, 48.28650835352867], [3.029770566828645, 48.285841287392934], [3.026173841403042, 48.28053907784034], [3.024963900419606, 48.276009138999655], [3.028346533708052, 48.2750411523276], [3.034433846656375, 48.273656629713386], [3.039878362606257, 48.27281545719324], [3.043632643430762, 48.272020556132226], [3.044682197610971, 48.27159908311009], [3.045469879539664, 48.27072242976915], [3.046195122889665, 48.26783125909786], [3.046149834088322, 48.266899136796795], [3.044260331022457, 48.26034067122589], [3.046003566622625, 48.2537160048418], [3.04756550376204, 48.24969616309244], [3.043081723287968, 48.24932088234353], [3.040427986993544, 48.249613341950095], [3.040366325929865, 48.25001825511847], [3.035868635073612, 48.250943758463784], [3.031461963016387, 48.24887916399273], [3.029094897438279, 48.24629925398094], [3.026207274836806, 48.24205034519273], [3.018566011385381, 48.235192866511866], [3.018924387944654, 48.23177458149728], [3.013838271052715, 48.22348656650058], [3.012811879416608, 48.22209832593543], [3.011314301568793, 48.22080279835678], [3.008565435350675, 48.21623304727507], [3.00824846878407, 48.21254846034834], [3.008513621521803, 48.21218672844865], [3.006456188881749, 48.209096997500495], [3.005159695747915, 48.20766550123541]])

In [6]:
def tweet_position_in_polygon(x, region):
    if x is not None and (region.contains(Polygon(x[0])) | region.contains(Point(x[0][0]))) :  
        return 1  
    else:
        return 0

In [7]:
tweets['in_paris']=tweets['bounding_box_coordinates'].apply(lambda x: tweet_position_in_polygon(x, polygon_paris))
tweets['in_idf']=tweets['bounding_box_coordinates'].apply(lambda x: tweet_position_in_polygon(x, polygon_idf))

In [8]:
users_idf = tweets.loc[tweets['in_idf']==1,'user_id'].unique().tolist()
users_paris = tweets.loc[tweets['in_paris']==1,'user_id'].unique().tolist()

In [9]:
print("Number of users in Ile-de-France : " + str(len(users_idf)))
print("Number of users in Paris : " + str(len(users_paris)))

Number of users in Ile-de-France : 18559
Number of users in Paris : 2839


# Export users names

On exporte les users-ids. On récupèrera ensuite l'historique de ces users-ids.

In [10]:
with open(path_to_export + "users_idf.json", "w") as outfile:
    outfile.write("\n".join(str(item) for item in users_idf))
with open(path_to_export + "users_paris.json", "w") as outfile:
    outfile.write("\n".join(str(item) for item in users_paris))